# NAME: HOSLEY, BRANDON

## OPER 785 Homework 4

In this assignment, you will be asked to construct an ANN to predict time-series data given past observed values.

**Instructions:** You have data from a time series, and you must construct a neural network to predict future values based on additional instructions below.

**Data:**

Mount your Colab drive to make data available for use.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Setup

Import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
#assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "hw4"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

*TO DO:* Load other packages you feel necessary to solve the problem at hand.

**Some plotting code that will be useful later**

You can adapt use this code or adapt it as you require. Alternately, you may code your own plotting algorithms as you see fit to meet the requirements in the sections below.

In [ ]:
def plot_series(series, y=None, y_pred=None, x_label="$t$", y_label="$x(t)$"):
    plt.plot(series, ".-")
    if y is not None:
        plt.plot(time_steps, y, "bx", markersize=10) # You will have to change "time_steps" to comply with your variable naming
    if y_pred is not None:
        plt.plot(time_steps, y_pred, "ro") # You will have to change "time_steps" to comply with your variable naming
    plt.grid(True)
    if x_label:
        plt.xlabel(x_label, fontsize=16)
    if y_label:
        plt.ylabel(y_label, fontsize=16, rotation=0)
    plt.hlines(0, 0, 100, linewidth=1)
    plt.axis([0, time_steps + 1, -1, 1]) # You will have to change "time_steps" to comply with your variable naming

def plot_learning_curves(loss, val_loss):
    plt.plot(np.arange(len(loss)) + 0.5, loss, "b.-", label="Training loss")
    plt.plot(np.arange(len(val_loss)) + 1, val_loss, "r.-", label="Validation loss")
    plt.gca().xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    plt.axis([1, 20, 0, 0.05])
    plt.legend(fontsize=14)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.grid(True)

def plot_ANN_history(df):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1) # sets verticle range to [0,1]
    plt.show



## Load data

Data is located in the file "OPER 785 hw4 data.csv"

**Format:** The data is 10000 sections (samples) of the time series. Each section (line) consists of 51 observations.

*To Do:* Load the data from the file. Keep the data in a variable you can refer to later (in other sections). Sectioning the data file will occur in each part of the assignment as needed for prediction requirements.

In [ ]:
# Your code here


*Optional
To Do:*

Look at your data in whatever way you may need or deem appropriate.

In [ ]:
# Your code here


## Part 1

For part 1, you will be predicting the next (single) value in the time series.

*To Do:*

1) section the data into the input and target (x and y),

2) section the data into training, testing, and validation sets (70%, 20%, 10%)

In [ ]:
# Your code here



*Optional To Do:*

Explore your time series

In [ ]:
# Your code here


*To Do:* Compute the mean square error of the validation set using a naive forecast.

In [ ]:
# Your code here



*To Do:* Construct a MLP model to make a linear prediction. Then calculate the mean square error of the validation set using your MLP. Discuss hyperparameter choices to include early stopping conditions selected. Experimentation is not required. However, you should explain your reasoning for hyperparameter settings.

In [ ]:
# Your code here



*To Do:* Plot your losses over the training epochs and discuss model fitness, overfit, and mitigation you may have used for the latter.

You may use the predefined functions "plot_learning_curves" or "plot_ANN_history" or code you have developed for the purpose as you please.

In [ ]:
# Your code here



*To Do:* Plot the first time series sample in your data set an include the truth value and the predicted value from your MLP model.

The predefined methods "plot_series" may be used for your convenience, or you may use your own code.

In [ ]:
# Your code here



*To Do:* Construct a Deep Recurrent Network to make predictions for the time series.

In [ ]:
# Your code here



*To Do:* Discuss your RNN. Points you must address: NN depth, width, hyperparameter settings, early stopping (if applicable). Your reasoning for each setting is important.

### Part 1 Analysis

*To Do:* Compare and discuss the results from you Deep RNN with the naive and linear forecast models.

## Part 2: Forecasting Multiple Steps Ahead

Going back to the original data you loaded from the file "OPER 785 hw4.csv," we are going to construct a recurrent neural network to predict 5 steps ahead.

*To Do:*

1) section the data into the input and target (x and y). Your y will consist of the last 5 observations in each series.

2) section the data into training, testing, and validation sets (70%, 20%, 10%)

In [ ]:
# Your code here



*To Do:*

Construct a Deep Recurrent Network capable of predicting multiple time steps ahead.

In [ ]:
# Your code here



*To Do:* Calculate the MSE for your Deep RNN. Discuss the difference between the MSE in predicting 10 values vs. the MSE in part 1, where only a single prediction was made. What makes them different?

In [ ]:
# Your code here

